# Run a Model against the Test-Data and Export its Battle Predictions

In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import pickle

In [11]:
def load(file_name, delimiter='|'):
    relative_path = '../data/'
    return pd.read_csv(relative_path+file_name, delimiter=delimiter)

Submission = load('01_raw/Submission.csv')
AvailablePokemons = load('01_raw/AvailablePokemons.csv')
BattleResults = load('01_raw/Battle_Results.csv')
Weakness_Pokemon = load('01_raw/Weakness_Pokemon.csv')
AllPokemons = load('01_raw/All_Pokemons.csv')

### Create Inference Data

In [7]:
def create_inference_data():
    """Return Same Format as Battle Results with additional PokemonID column"""
    # Repeat every row 6 times
    merged = pd.DataFrame(np.repeat(AvailablePokemons.values,6,axis=0), columns=AvailablePokemons.columns)
    # Drop PokemonID from Submission
    Submission_dropped = Submission.drop(columns=['SelectedPokemonID'])
    # Concat with Submission
    merged = merged.apply(lambda row: pd.concat([row, Submission_dropped.iloc[row.name%6]]), axis=1)
    return merged

In [8]:
inference_data = create_inference_data()

### Import Model and Pipeline

In [21]:
model = keras.models.load_model('../data/03_models/nn_model')

In [23]:
with open('../data/03_models/nn_pipeline.pickle', 'rb') as handle:
    pipeline = pickle.load(handle)

AttributeError: Can't get attribute 'AddPokemonTypes' on <module '__main__'>

### Run Model against Inference

In [9]:
x_inference = pipeline.transform(inference_data)
inference_predictions = model.predict(x_inference)
inference_predictions

### Run Model against Test Data

In [12]:
from sklearn.metrics import mean_squared_error

In [15]:
BattleResults_Test = load('02_intermediate/Battle_Results_Test.csv', delimiter=',')

In [17]:
def RelativeResult(X):
    HP = X.apply(lambda row: row["HP_1"] if row["BattleResult"] > 0 else row["HP_2"], axis=1).values
    relativized = X["BattleResult"].values / HP
    return relativized

test_labels = RelativeResult(BattleResults_Test)

In [18]:
x_test = pipeline.fit_transform(BattleResults_Test)
test_predictions = model.predict(x_test)
lin_mse = mean_squared_error(test_labels, test_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

1.0537348322623872

### Export

In [39]:
def export(predictions):
    table = np.empty((len(AvailablePokemons), 7))
    for i in range(len(AvailablePokemons)):
        table[i, 0] = i
        table[i, 1:] = predictions[i*6: (i+1)*6]
    df = pd.DataFrame(data=table, columns=['SelectedPokemonID']+list(Submission.Name_2.values)); output
    return df.astype({'SelectedPokemonID': int})
    
output = export(inference_predictions)
output

,SelectedPokemonID,Caterpie,Golem,Krabby,Mewtwo,Raichu,Venusaur
0,0,-0.673347,0.876185,-0.001903,0.044104,-0.088475,-0.505112
1,1,0.331356,-0.544088,-0.307657,-0.095056,0.942036,0.677011
2,2,-0.015797,0.358002,0.620032,-0.089927,0.357616,-0.313250
3,3,-0.178182,0.111369,0.016455,0.698512,-0.363971,-0.096498
4,4,0.646152,-0.913883,-0.551524,0.192226,0.147515,0.325275
...,...,...,...,...,...,...,...
1435,1435,-0.070775,-0.844019,-0.052456,0.827367,-0.387723,-0.636623
1436,1436,-0.721260,0.961378,-0.101890,0.150286,-0.840004,0.786808
1437,1437,-0.712511,0.375991,-0.128202,-0.922079,0.623179,0.227376
1438,1438,-0.306187,0.583784,-0.799536,-0.608873,0.365639,-0.058822


In [42]:
output.to_csv(path_or_buf='../../data/07_model_output/Random/inference_random.csv')